<a href="https://colab.research.google.com/github/MunJinSeo/MyProject/blob/master/daniel021_friends_electra_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 영어(Friends) ELECTRA를_이용한_감정분석기_학습 (Pytorch + HuggingFace)
# Colab 에서 개발 및 실행
# Github : https://github.com/MunJinSeo/MyProject/
<br>

## References 1
- 김희규님의 "HuggingFace KoElectra로 NSMC 감성분석 Fine-tuning해보기"<br>
https://heegyukim.medium.com/huggingface-koelectra%EB%A1%9C-nsmc-%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0-1a23a0c704af

- 이지원님의 Github : nlp_emotion_classification <br>
https://github.com/jiwonny/nlp_emotion_classification

## 사용모델 ELECTRA
- 한국어 : 박장원님의 KoELECTRA 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA
- 영어 : 구글 ELECTRA 사용<br>
https://huggingface.co/google/electra-small-discriminator<br>
https://github.com/google-research/electra

## Dataset (학습에는 train만 사용함, test는 감점대상)
- 한국어 : 네이버 영화 리뷰 데이터셋 <br>
https://github.com/e9t/nsmc
- 영어 : Freinds <br>
http://doraemon.iis.sinica.edu.tw/emotionlines/

## References 2
- https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP
- https://blog.naver.com/horajjan/221739630055
<br>@전처리 관련@<br>
- https://github.com/YongWookHa/kor-text-preprocess
- https://github.com/likejazz/korean-sentence-splitter
- https://github.com/lovit/soynlp
<br>@@<br>
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249


## 기타/유의사항
- 반드시 GPU로 실행 - Colab무료환경에서 1epoch 당 약 10~20분 소요<br>
- CUDA(GPU) 메모리 오버되는경우 학습시 Batch size 줄여서 해볼것
- 학습용 데이터 파일은 사전에 업로드 할것

# 필요 lib 설치

In [ ]:
# lib 설치
!pip install transformers
!pip install torch

     |████████████████████████████████| 1.5MB 6.5MB/s 
     |████████████████████████████████| 890kB 30.2MB/s 
     |████████████████████████████████| 2.9MB 35.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b3ad5be1e030e18f2299f119b13de857891dc629c6da16d0611fb865f1eec0aa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#!pip install kss
#!pip install konlpy
#!pip install sentencepiece
#!pip install soynlp

In [ ]:
# (미사용) Colab TPU 사용을 위해 설치
#--!pip install torch_xla
#--#@param ["1.5" , "20200325", "nightly"]
#VERSION = "1.7"
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version $VERSION

# Friends 데이터셋 처리

In [ ]:
#Friends 데이터셋 처리
#zip파일로 업로드시 아래 사용, Colab에 직접 올려서 사용함
#!unzip friends_json.zip


In [ ]:
# 학습에는 train만 사용함, test는 감점대상
!head friends_train.json
!head friends_test.json
!head friends_dev.json
# !head en_data.csv #별도 kaggle에서 받아서 사용 https://www.kaggle.com/c/english-sa-competition-bdc101

[
    [
        {
            "speaker": "Chandler",
            "utterance": "also I was the point person on my company\u0092s transition from the KL-5 to GR-6 system.",
            "emotion": "neutral",
            "annotation": "4100000"
        },
        {
            "speaker": "The Interviewer",
[
    [
        {
            "speaker": "Mark",
            "utterance": "Why do all you\u0092re coffee mugs have numbers on the bottom?",
            "emotion": "surprise",
            "annotation": "2000030"
        },
        {
            "speaker": "Rachel",
[
    [
        {
            "speaker": "Phoebe",
            "utterance": "Oh my God, he\u0092s lost it. He\u0092s totally lost it.",
            "emotion": "non-neutral",
            "annotation": "0002120"
        },
        {
            "speaker": "Monica",


# 필요 모듈 import

In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
# (미사용) TPU 사용을 위해 필요
#import torch_xla
#import torch_xla.core.xla_model as xm

In [ ]:
import json
import numpy as np

In [ ]:
#import kss
import re
#from soynlp.normalizer import *

In [ ]:
# GPU or CPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('No GPU available, using the CPU instead.')

# (미사용) TPU
#device = xm.xla_device()

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


# 데이터셋 처리 (Dataset Calss / 전처리)
(train data와 과제 sample data형식이 다르고, encoding이 다르기 때문에 분리 처리)

In [ ]:
class Friends_Dataset(Dataset):
  # Json파일을 DataFrame으로 변환 함수
  def jsonToDataFrame(self, file_name):
    with open(file_name, encoding = 'utf-8', mode = 'r') as file:
      json_array = json.load(file)

    result = pd.DataFrame.from_dict(json_array[0])
   
    is_first = True
    for array in json_array:
      if is_first:
        is_first = False
        continue
      
      temp_df = pd.DataFrame.from_dict(array)
      result = result.append(temp_df, ignore_index = True)

    return result

  def __init__(self, csv_file, ftype):
    # train data와 sample data 각각 처리

    # 초기 전처리 1, 아래쪽 전처리 2 로 clean_text() 펑션 분리함 (학습 할때 line별 전처리 후 사용됨)
    # --- 전처리 1 start ------------------------
    self.emotion_dic = {'neutral':0,'surprise':1,'fear':2,'non-neutral':3,'joy':4,'sadness':5,'anger':6,'disgust':7}

    if ftype == 'train':
      #1 speaker, 2 utterance(표현문장), 3 emotion(감정분류값), 4 annotation
      self.dataset = self.jsonToDataFrame(file_name=csv_file) #.dropna(axis=0)

    elif ftype == 'sample':
      #1 id, 2 i_dialog, 3 i_utterance, 4 speaker, 5 utterance
      #self.dataset = pd.read_csv(csv_file, sep=',', encoding = 'unicode_escape')
      self.dataset = pd.read_csv(csv_file, sep=',', encoding = 'utf-8')
      # 뒤쪽 컬럼 label 값이 없으므로 기본값으로 추가함
      #0 idx, 1 id, 2 i_dialog, 3 i_utterance, 4 speaker, 5 utterance(표현문장),6 emotion, 7 Predicted 
      self.dataset.insert(5,'emotion',0) # 기본값 셋팅
      self.dataset.insert(6,'Predicted',0) # 기본값 셋팅
      #print(self.dataset)
    else:
      self.dataset = pd.read_csv(csv_file, sep=',')


    # for idx, document in self.dataset.iterrows():
    #   print(idx, document)
    self.sub1 = re.compile('[^ .?!/@$%~|0-9|a-zㅣA-Z]+') # 영어와 띄어쓰기, 특수기호 일부를 제외한 모든 글자제거
    self.sub2 = re.compile('[\s]+')  # white space duplicate
    self.sub3 = re.compile('[\.]+')  # full stop duplicate

    #중복되는 문장 제거
    if ftype == 'sample':
      print('과제 제출용은 중복 문장 제거하면 안됨')
    else:
      # 중복제거: utterance가 텍스트 내용의 title명이다
      self.dataset.drop_duplicates(subset=['utterance'], inplace=True)

    # dataset 확인
    print(self.dataset.describe())
    print(self.dataset)

    # tokenizer
    self.tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    #cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    #cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    #if len(ssStr) > 1:
    cleaned = "[CLS] " + cleaned + " [SEP]"
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):
    # 행번호별 컬럼 지정하여 할당: 0 idx, 1 speaker, 2 utterance, 3 emotion, 4 annotation
    #row = self.dataset.iloc[idx, 1:4].values  # idx번째 행의 첫번째 컬럼 0을 제외하고 1~3컬럼
    #print(row[0], row[1])
    #text = self.clean_text( txt= str(row[0]) ) # 전처리 2 : clean_text()
    #y = row[1] #숫자만 가능함 

    text = self.dataset['utterance'][idx]
    tmp_y = self.dataset['emotion'][idx]
    # emotion 값을 숫자로 치환
    y = 0
    if tmp_y in self.emotion_dic.keys() :
      y = self.emotion_dic[tmp_y]
    else :
      y = 0
    #print("[%s][%d]" % (tmp_y,y) )

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
# 학습에는 train만 사용함, test는 감점대상
train_dataset = Friends_Dataset("friends_train.json","train")

#test데이터는 감점사항으로 제외함
#train2_dataset = Friends_Dataset("friends_test.json","train")
#train_dataset.dataset = train_dataset.dataset.append(train2_dataset.dataset, ignore_index = True)
#print("train_dataset.dataset==ALL=\n", train_dataset.dataset)
#print("train_dataset.dataset.head()==\n", train_dataset.dataset.head())

test_dataset = Friends_Dataset("friends_test.json","train")

sample_dataset = Friends_Dataset("en_data.csv","sample")
#print("sample_dataset.dataset.head()==\n", sample_dataset.dataset.head())

       speaker        utterance  emotion annotation
count     9291             9291     9291       9291
unique     259             9291        8        412
top       Joey  Oh. Oh wait no.  neutral    5000000
freq      1395                1     4116       1173
               speaker  ... annotation
0             Chandler  ...    4100000
1      The Interviewer  ...    5000000
2             Chandler  ...    5000000
3      The Interviewer  ...    5000000
4             Chandler  ...    2000030
...                ...  ...        ...
10555             Joey  ...    2000003
10556         Chandler  ...    3000011
10557             Ross  ...    2100011
10558             Joey  ...    0000050
10560             Joey  ...    1200020

[9291 rows x 4 columns]



       speaker                              utterance  emotion annotation
count     2505                                   2505     2505       2505
unique      98                                   2505        8        311
top       Joey  So why dont you give me your number?  neutral    5000000
freq       388                                      1     1159        328
       speaker  ... annotation
0         Mark  ...    2000030
1       Rachel  ...    2100011
2       Rachel  ...    3000020
3         Ross  ...    5000000
4        Frank  ...    1300010
...        ...  ...        ...
2759    Phoebe  ...    5000000
2760    Monica  ...    1400000
2761    Phoebe  ...    3100100
2762    Monica  ...    5000000
2763  Chandler  ...    4000100

[2505 rows x 4 columns]
       speaker             utterance  emotion annotation
count     1090                  1090     1090       1090
unique      47                  1090        8        234
top       Ross  The week after that?  neutral    5000000
freq

In [ ]:
# tmpstr = 'Come on.  Hello?  I?m sorry you have the wrong number.   Okay, I?ll call you later dad. I love you.'
# print( train_dataset.clean_text( txt = tmpstr) )

In [ ]:
# test_emotion = 'joy'
# if test_emotion in train_dataset.emotion_dic.keys() :
#   print( train_dataset.emotion_dic[test_emotion] )
# else :
#   print(0)

In [ ]:
# print( train_dataset.__getitem__(11790) )

In [ ]:
# print( sample_dataset.__getitem__(10) )

# 모델 생성 (Create Model)

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator", num_labels=8).to(device)
#model = ElectraForSequenceClassification.from_pretrained('google/electra-small-generator', num_labels=8)
#model.cuda()

# 한번 실행해보기
#text, attention_mask, y = train_dataset[0]
#model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d

In [ ]:
try:
  model.load_state_dict(torch.load("model.pt"))
  #model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/model_daniel021_friends_electra_small_epoch8.pt"))
except:
  print("error - model.load_state_dict(torch.load('model.pt'))")
else:
  print("success - model.load_state_dict(torch.load('model.pt'))")

success - model.load_state_dict(torch.load('model.pt'))


In [ ]:
# 모델 레이어 보기
model

# 학습(Learn) - train파일만 사용함(test제외)

In [ ]:
# google/electra-small-discriminator
#epochs = 16
#batch_size = 64 #CUDA(GPU) 메모리 오버되는경우 size 줄여서 해볼것

# google/electra-base-discriminator
epochs = 16
batch_size = 16 #CUDA(GPU) 메모리 오버되는경우 size 줄여서 해볼것

In [ ]:
optimizer = AdamW(model.parameters(), 
                  lr=1e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                  )
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
losses = []
accuracies = []

# 그래디언트 초기화
# model.zero_grad()

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  # 훈련모드로 변경
  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward() # Backward 수행으로 그래디언트 계산
    #xm.optimizer_step(optimizer, barrier=True)  # TPU 사용시 코드
    optimizer.step() # 그래디언트를 통해 가중치 파라미터 업데이트
    #model.zero_grad() # 그래디언트 초기화

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    #print("y_batch====\n",y_batch)
    #print("predicted====\n",predicted)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 29.380662936717272 Accuracy: tensor(0.9075, device='cuda:0')
Batch Loss: 59.06011170335114 Accuracy: tensor(0.9031, device='cuda:0')
Batch Loss: 88.05364321358502 Accuracy: tensor(0.9042, device='cuda:0')
Batch Loss: 122.67656613700092 Accuracy: tensor(0.8994, device='cuda:0')
Batch Loss: 153.9791846703738 Accuracy: tensor(0.8999, device='cuda:0')
Batch Loss: 184.9016562383622 Accuracy: tensor(0.9002, device='cuda:0')
Batch Loss: 218.4831743631512 Accuracy: tensor(0.8998, device='cuda:0')

Train Loss: 229.9805915299803 Accuracy: tensor(0.9003, device='cuda:0')


Batch Loss: 23.411132652312517 Accuracy: tensor(0.9294, device='cuda:0')
Batch Loss: 48.632437348365784 Accuracy: tensor(0.9269, device='cuda:0')
Batch Loss: 75.6391849219799 Accuracy: tensor(0.9208, device='cuda:0')
Batch Loss: 100.75911046750844 Accuracy: tensor(0.9211, device='cuda:0')
Batch Loss: 128.65358585119247 Accuracy: tensor(0.9185, device='cuda:0')
Batch Loss: 153.78550720959902 Accuracy: tensor(0.9178, device='cuda:0')
Batch Loss: 180.97218305245042 Accuracy: tensor(0.9166, device='cuda:0')

Train Loss: 189.64467968046665 Accuracy: tensor(0.9169, device='cuda:0')


Batch Loss: 19.23243652936071 Accuracy: tensor(0.9456, device='cuda:0')
Batch Loss: 40.56486351694912 Accuracy: tensor(0.9400, device='cuda:0')
Batch Loss: 62.54419625643641 Accuracy: tensor(0.9373, device='cuda:0')
Batch Loss: 84.30779746361077 Accuracy: tensor(0.9367, device='cuda:0')
Batch Loss: 107.64409747347236 Accuracy: tensor(0.9355, device='cuda:0')
Batch Loss: 127.19162307213992 Accuracy: tensor(0.9374, device='cuda:0')
Batch Loss: 151.84586540888995 Accuracy: tensor(0.9346, device='cuda:0')

Train Loss: 160.83783608768135 Accuracy: tensor(0.9351, device='cuda:0')


Batch Loss: 17.444781593978405 Accuracy: tensor(0.9450, device='cuda:0')
Batch Loss: 34.10211717709899 Accuracy: tensor(0.9444, device='cuda:0')
Batch Loss: 49.635076221544296 Accuracy: tensor(0.9465, device='cuda:0')
Batch Loss: 74.84025323251262 Accuracy: tensor(0.9397, device='cuda:0')
Batch Loss: 90.1595876053907 Accuracy: tensor(0.9424, device='cuda:0')
Batch Loss: 109.56426458386704 Accuracy: tensor(0.9405, device='cuda:0')
Batch Loss: 127.2276137615554 Accuracy: tensor(0.9406, device='cuda:0')

Train Loss: 135.93686838028952 Accuracy: tensor(0.9399, device='cuda:0')


Batch Loss: 15.727003321051598 Accuracy: tensor(0.9550, device='cuda:0')
Batch Loss: 31.859436224214733 Accuracy: tensor(0.9522, device='cuda:0')
Batch Loss: 47.92349840328097 Accuracy: tensor(0.9513, device='cuda:0')
Batch Loss: 66.50131967104971 Accuracy: tensor(0.9494, device='cuda:0')
Batch Loss: 82.04884139355272 Accuracy: tensor(0.9491, device='cuda:0')
Batch Loss: 99.78529685828835 Accuracy: tensor(0.9480, device='cuda:0')
Batch Loss: 116.70232293289155 Accuracy: tensor(0.9479, device='cuda:0')

Train Loss: 122.18205303978175 Accuracy: tensor(0.9480, device='cuda:0')


Batch Loss: 12.812323292717338 Accuracy: tensor(0.9619, device='cuda:0')
Batch Loss: 27.82873457437381 Accuracy: tensor(0.9556, device='cuda:0')
Batch Loss: 41.58078985661268 Accuracy: tensor(0.9569, device='cuda:0')
Batch Loss: 55.30111645627767 Accuracy: tensor(0.9589, device='cuda:0')
Batch Loss: 70.0999566456303 Accuracy: tensor(0.9573, device='cuda:0')
Batch Loss: 84.54149791458622 Accuracy: tensor(0.9559, device='cuda:0')
Batch Loss: 100.12534970417619 Accuracy: tensor(0.9542, device='cuda:0')

Train Loss: 106.04557873029262 Accuracy: tensor(0.9541, device='cuda:0')


Batch Loss: 13.946621155366302 Accuracy: tensor(0.9525, device='cuda:0')
Batch Loss: 26.820435866247863 Accuracy: tensor(0.9566, device='cuda:0')
Batch Loss: 40.07746725110337 Accuracy: tensor(0.9567, device='cuda:0')
Batch Loss: 51.19882704084739 Accuracy: tensor(0.9586, device='cuda:0')
Batch Loss: 64.5420282012783 Accuracy: tensor(0.9584, device='cuda:0')
Batch Loss: 79.4708350431174 Accuracy: tensor(0.9573, device='cuda:0')
Batch Loss: 93.73114714585245 Accuracy: tensor(0.9569, device='cuda:0')

Train Loss: 97.74719863990322 Accuracy: tensor(0.9571, device='cuda:0')


Batch Loss: 11.087331159040332 Accuracy: tensor(0.9656, device='cuda:0')
Batch Loss: 20.797400156967342 Accuracy: tensor(0.9644, device='cuda:0')
Batch Loss: 32.759778483305126 Accuracy: tensor(0.9617, device='cuda:0')
Batch Loss: 45.88531629857607 Accuracy: tensor(0.9603, device='cuda:0')
Batch Loss: 57.1636138537433 Accuracy: tensor(0.9609, device='cuda:0')
Batch Loss: 68.33246290241368 Accuracy: tensor(0.9616, device='cuda:0')
Batch Loss: 81.29238559026271 Accuracy: tensor(0.9613, device='cuda:0')

Train Loss: 85.61059090634808 Accuracy: tensor(0.9614, device='cuda:0')


In [ ]:
losses, accuracies

# 테스트 데이터셋 정확도 확인하기

In [ ]:
# 평가모드로 변경
model.eval()

#test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

# 모델 저장하기

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "model.pt")

# 과제용 데이터 예측 및 맵핑

In [ ]:
#과제용 데이터 예측
# 데이터 로딩
batchSize = 16
sample_loader = DataLoader(sample_dataset, batch_size=batchSize, shuffle=False)

sample_result = sample_dataset.dataset.copy(deep=True)
print(sample_result)

#평가모드로 변경
model.eval()

idx_s = 0
idx_e = 0

emotion_strDic = {0:'neutral',1:'surprise',2:'fear',3:'non-neutral',4:'joy',5:'sadness',6:'anger',7:'disgust'}

for input_ids_batch, attention_masks_batch, y_batch in tqdm(sample_loader):
  #y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)

  rsList = list(map(int, predicted)) # 결과를 한번에 저장하기 위해 LIST로 변환 처리
  global idx_s, idx_e
  idx_e += len(rsList) #해당 배치구간내에 index 끝값 계산
  #print("index==", idx_s, idx_e)
  
  sample_result['emotion'][idx_s : idx_e] = rsList
  rsPred = [emotion_strDic[xx] for xx in rsList] # 숫자를 감정문자로 치환
  sample_result['Predicted'][idx_s : idx_e] = rsPred  #배치구간을 한번에 업데이트
  idx_s += len(rsList) #해당 배치구간내에 index 시작값은 윗줄 처리 후 증가

print(sample_result)


In [ ]:
#torch.cuda.empty_cache() #GPU 캐쉬 데이터 삭제

# 결과 파일 저장

In [ ]:
# 주어진 데이터의 결과를 파일로 저장
#sample_csv = sample_result.to_csv('sample.csv')
sample_csv = sample_result.to_csv('sample.csv',sep=',',na_rep='NaN', columns=['id','Predicted'],index=False)

In [ ]:
# 파일을 PC로 다운로드 하기
from google.colab import files
files.download('sample.csv')


In [ ]:
# 나의 구글Drive 연결하기
#from google.colab import drive
#drive.mount('/content/drive')